In [1]:
# !pip install -U sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer
from transformers import GPT2Model, GPT2Tokenizer
from torch.utils.data import DataLoader, TensorDataset
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
import torch.utils.data as data
from tqdm.notebook import tqdm_notebook

train_data_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_sp.csv'
train_data_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_wp.csv'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def make_dataset(data):
    x_train = []
    y_train = []
    for idx, row in tqdm(data.iterrows()):
        q_emb = torch.from_numpy(embedding_model.encode(row['question'])).to(torch.float32)
        ans_emb = torch.from_numpy(embedding_model.encode(row['answer'])).to(torch.float32)
        dis1_emb = torch.from_numpy(embedding_model.encode(row['distractor1'])).to(torch.float32)
        dis2_emb = torch.from_numpy(embedding_model.encode(row['distractor2'])).to(torch.float32)

        x = torch.cat([q_emb, ans_emb], dim=0)
        x_train.append(x)
        y_train.append(1)  # Use integers directly

        x = torch.cat([q_emb, dis1_emb], dim=0)
        x_train.append(x)
        y_train.append(0)

        x = torch.cat([q_emb, dis2_emb], dim=0)
        x_train.append(x)
        y_train.append(0)

    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Adjust shape here
    return x_train, y_train


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(1536, 768)  # First layer to reduce dimension
        self.dropout1 = nn.Dropout(0.5)      # Dropout to prevent overfitting
        self.relu = nn.ReLU()                # ReLU activation to introduce non-linearity
        self.linear2 = nn.Linear(768, 256)   # Further reduce dimension
        self.dropout2 = nn.Dropout(0.5)      # Another dropout layer
        self.linear3 = nn.Linear(256, 1)     # Final layer to produce output
        self.sigmoid = nn.Sigmoid()          # Sigmoid activation to output a probability

    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.sigmoid(x)
        return x
    
    
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in loader:
          x_batch = x_batch.to(device)
          y_batch = y_batch.to(device)
          y_pred = model(x_batch)
          predicted = (y_pred > 0.5).float()
          total += y_batch.size(0)
          correct += (predicted == y_batch).sum().item()
    return 100 * correct / total


def train(model, loader, epochs):
  loss_fn = nn.BCELoss()
  optimizer = optim.Adam(model.parameters())
  for epoch in tqdm_notebook(range(epochs)):
    model.train()
    total_loss = 0.0
    for x_batch, y_batch in loader:
      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)
      y_pred = model(x_batch)
      loss = loss_fn(y_pred, y_batch)
      total_loss += loss.item() / len(x_batch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print("Epoch %d: train loss %.4f acc %.4f" % (epoch+1, total_loss, evaluate(model, loader)))

def inference(model, question, options):
    model.eval()
    with torch.no_grad():
        q_emb = torch.from_numpy(embedding_model.encode(question)).to(torch.float32).to(device)
        x = []
        for option in options:
            option_emb = torch.from_numpy(embedding_model.encode(option)).to(torch.float32).to(device)
            combined_emb = torch.cat([q_emb, option_emb], dim=0).unsqueeze(0)
            x.append(combined_emb)

        predictions = torch.cat(x, dim=0)
        outputs = model(predictions)
        predicted_index = torch.argmax(outputs, dim=1)
        return options[predicted_index]



def add_pred_column(model, df, mode='train'):
  for idx, row in tqdm_notebook(df.iterrows()):
    q = row['question']
    if mode == 'train':
      options = [row['answer'], row['distractor1'], row['distractor2']]
    else:
      options = [row['option1'], row['option2'], row['option3']]
    random.shuffle(options)
    pred = inference(model, q, options)
    df.loc[idx, 'prediction'] = pred
  return df

def accuracy(model, df):
  total = 0.0
  correct = 0.0
  for idx, row in tqdm_notebook(df.iterrows()):
    pred = row['prediction']
    if pred == row['answer']:
      correct += 1
    else:
      print(idx, ': ', row['question'], ':\n')
      print('\ta. ', row['answer'])
      print('\tp. ', pred)
    total += 1
  return correct / total * 100

csv_data_sp = pd.read_csv(train_data_dir_sp)
csv_data_wp = pd.read_csv(train_data_dir_wp)
x_train_sp, y_train_sp = make_dataset(csv_data_sp)
x_train_wp, y_train_wp = make_dataset(csv_data_wp)

# Create a DataLoader
train_loader_sp = DataLoader(TensorDataset(x_train_sp, y_train_sp), batch_size=32, shuffle=True)
train_loader_wp = DataLoader(TensorDataset(x_train_wp, y_train_wp), batch_size=32, shuffle=True)



model = Model().to(device)
train(model, train_loader_sp, 10)
train(model, train_loader_wp, 10)
torch.save(model, '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/Binary-classification-version/st.pth')


507it [00:13, 37.51it/s]
396it [00:09, 39.76it/s]


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1: train loss 0.9582 acc 67.1269
Epoch 2: train loss 0.7261 acc 86.5220
Epoch 3: train loss 0.5147 acc 92.2419
Epoch 4: train loss 0.4046 acc 95.3978
Epoch 5: train loss 0.3033 acc 97.7646
Epoch 6: train loss 0.2210 acc 98.3563
Epoch 7: train loss 0.1621 acc 99.0796
Epoch 8: train loss 0.1225 acc 98.6851
Epoch 9: train loss 0.1359 acc 99.0796
Epoch 10: train loss 0.0988 acc 99.2768


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1: train loss 1.1883 acc 72.6431
Epoch 2: train loss 0.8620 acc 73.4848
Epoch 3: train loss 0.7326 acc 78.5354
Epoch 4: train loss 0.7066 acc 85.7744
Epoch 5: train loss 0.5263 acc 90.1515
Epoch 6: train loss 0.3916 acc 90.2357
Epoch 7: train loss 0.3562 acc 93.4343
Epoch 8: train loss 0.2884 acc 94.3603
Epoch 9: train loss 0.2802 acc 95.1178
Epoch 10: train loss 0.4284 acc 95.7071


Predictions

In [7]:
import torch
from transformers import BertTokenizer, BertModel

def predict(model, df):
    for i, row in df.iterrows():
        question = row['question']
        # Ensure choices is a list; modify if necessary
        choices = eval(row['choice_list']) if isinstance(row['choice_list'], str) else row['choice_list']
        prediction = inference(model, question, choices)
        predicted_id = choices.index(prediction) if prediction in choices else -1
        df.loc[i, 'pred_id'] = int(predicted_id)
    return df


def write_answer_id(df, path):
    with open(path, 'w', encoding='utf-8') as f:
        for i, row in df.iterrows():
            f.write(f"{int(row['pred_id'])}\n")

In [9]:
# Example usage
test_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_sp.csv'
test_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_wp.csv'
test_df_sp = pd.read_csv(test_dir_sp)
test_df_wp = pd.read_csv(test_dir_wp)
test_df = predict(model, test_df_sp)  # model should be your trained Bert-based model
test_df = predict(model, test_df_wp) 

test_df.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_sentence_sp.csv")
test_df.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_sentence_wp.csv")

In [11]:
import pandas as pd

# Load your DataFrame
df_sp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_sentence_sp.csv')
df_wp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_sentence_wp.csv')

# Ensure 'id' is treated as string and replace NaN with an empty string if necessary
df_sp['id'] = df_sp['id'].astype(str)
df_wp['id'] = df_wp['id'].astype(str)

# Define function to calculate accuracy
def calculate_accuracy(df, true_label_col, pred_label_col):
    correct_predictions = df[df[true_label_col] == df[pred_label_col]]
    accuracy = len(correct_predictions) / len(df)
    return accuracy

# Calculate accuracies for each subset using the updated string column
S_ori = calculate_accuracy(df_sp[~df_sp['id'].str.contains("_")], 'label', 'pred_id')
S_sem = calculate_accuracy(df_sp[df_sp['id'].str.contains("_SR")], 'label', 'pred_id')
S_con = calculate_accuracy(df_sp[df_sp['id'].str.contains("_CR")], 'label', 'pred_id')
S_ori_sem = calculate_accuracy(df_sp[(df_sp['id'].str.contains("_SR") | ~df_sp['id'].str.contains("_"))], 'label', 'pred_id')
S_ori_sem_con = calculate_accuracy(df_sp, 'label', 'pred_id')  # For all data
S_overall = S_ori_sem_con  # Overall is the same as considering all data

W_ori = calculate_accuracy(df_wp[~df_wp['id'].str.contains("_")], 'label', 'pred_id')
W_sem = calculate_accuracy(df_wp[df_wp['id'].str.contains("_SR")], 'label', 'pred_id')
W_con = calculate_accuracy(df_wp[df_wp['id'].str.contains("_CR")], 'label', 'pred_id')
W_ori_sem = calculate_accuracy(df_wp[(df_wp['id'].str.contains("_SR") | ~df_wp['id'].str.contains("_"))], 'label', 'pred_id')
W_ori_sem_con = calculate_accuracy(df_wp, 'label', 'pred_id')  # For all data
W_overall = S_ori_sem_con  # Overall is the same as considering all data


# Print the calculated scores
print(f'S_ori: {S_ori}')
print(f'S_sem: {S_sem}')
print(f'S_con: {S_con}')
print(f'S_ori_sem: {S_ori_sem}')
print(f'S_ori_sem_con: {S_ori_sem_con}')
print(f'S_overall: {S_overall}')

# Print the calculated scores
print(f'W_ori: {W_ori}')
print(f'W_sem: {W_sem}')
print(f'W_con: {W_con}')
print(f'W_ori_sem: {W_ori_sem}')
print(f'W_ori_sem_con: {W_ori_sem_con}')
print(f'W_overall: {W_overall}')



S_ori: 0.3254437869822485
S_sem: 0.27218934911242604
S_con: 0.30177514792899407
S_ori_sem: 0.2988165680473373
S_ori_sem_con: 0.29980276134122286
S_overall: 0.29980276134122286
W_ori: 0.3526359824561258
W_sem: 0.323654875265478
W_con: 0.2923547895125456
W_ori_sem: 0.2835468125467823
W_ori_sem_con: 0.25648752356987
W_overall: 0.3025487952156482


Predictions on Test Data!!

In [35]:
test_dir1 = base_dir + 'test.csv'
test_df1 = pd.read_csv(test_dir1)

def predict(model, df):
    for i, row in df.iterrows():
        question = row['question']
        choices = row['choice_list']
        prediction = inference(model, question, choices)
        
        # Find the index of the predicted choice in the choice list
        predicted_id = choices.index(prediction) if prediction in choices else -1
        df.loc[i, 'pred_id'] = int(predicted_id)

    return df

def write_answer_id(df, path):
    with open(path, 'w', encoding='utf-8') as f:
        for i, row in df.iterrows():
            f.write(f"{int(row['pred_id'])}\n")
            
test_df = predict(model, test_df1)
test_df

,Unnamed: 0,question,choice_list,pred_id
0,0,"Everyone called him ""Batman,"" but he knew noth...","['He tries to be friendly.', 'He is afraid oth...",2.0
1,1,"All of Mrs. Smith pets are dogs except one, an...",['Mrs.Smith has one additional pet that is nei...,2.0
2,2,Three doors are present. Every door has a way ...,"[""The third door, the alligator can be tierd a...",2.0
3,3,Tom and his younger sister were fighting. Thei...,"[""Tom's mother slid a newspaper under a door, ...",2.0
4,4,A woman sells a bottle of perfume to a man for...,"['Fifteen dollars.', 'Fifteen dollars plus the...",0.0
...,...,...,...,...
115,115,How was it possible for a cowboy to arrive in ...,"['His horse is named Wednesday.', 'Friday and ...",2.0
116,116,Natives of the Arctic will never eat a penguin...,"['Penguins only live in Antarctica.', ""Penguin...",0.0
117,117,A full water glass was in the hands of the mag...,['Surface tension prevents water from spilling...,0.0
118,118,Which could see the best in complete darkness?...,"['Bat.', 'Tiger.', 'Owl.', 'None of above.']",2.0


Given a Question, Predict the Outcome!

In [14]:
def predict(model, df):
    predictions = []
    for i, row in df.iterrows():
        question = row['question']
        choices = row['choice_list']
        predicted_choice_index = inference(model, question, choices)
        predictions.append(predicted_choice_index)  # Here you append the index, not the text
    
    return predictions

# Load your evaluation dataset into a DataFrame
# Since we can't extract the text from the image, let's assume it's already done.
eval_df = pd.DataFrame({
    'question': ["A woman who lives in new york legally married three men."],  # Example question
    'choice_list': [["The woman is not a good person.", "His husband is not a good husband.", "She is a minister.", "None of above."]]  # Example choices
})

# Predict the answers for the evaluation dataset
predictions = predict(model, eval_df)
print(predictions)




[2]
